In [1]:
import os

In [6]:
os.chdir('D:/dataset/text summary cosine sim/textrank_text_summarization')
data_path = 'D:/dataset/text summary cosine sim/textrank_text_summarization'

In [3]:
pwd()

'D:\\dataset\\text summary cosine sim\\textrank_text_summarization'

In [4]:
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt') # one time execution
import re

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\shri1\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
import io

In [8]:
df = pd.read_csv('tennis_articles_v4.csv')

In [9]:
df

,article_id,article_text,source
0,1,Maria Sharapova has basically no friends as te...,https://www.tennisworldusa.org/tennis/news/Mar...
1,2,"BASEL, Switzerland (AP), Roger Federer advance...",http://www.tennis.com/pro-game/2018/10/copil-s...
2,3,Roger Federer has revealed that organisers of ...,https://scroll.in/field/899938/tennis-roger-fe...
3,4,Kei Nishikori will try to end his long losing ...,http://www.tennis.com/pro-game/2018/10/nishiko...
4,5,"Federer, 37, first broke through on tour over ...",https://www.express.co.uk/sport/tennis/1036101...
5,6,Nadal has not played tennis since he was force...,https://www.express.co.uk/sport/tennis/1037119...
6,7,"Tennis giveth, and tennis taketh away. The end...",http://www.tennis.com/pro-game/2018/10/tennisc...
7,8,Federer won the Swiss Indoors last week by bea...,https://www.express.co.uk/sport/tennis/1038186...


In [11]:
# split the the text in the articles into sentences
sentences = []
for s in df['article_text']:
    sentences.append(sent_tokenize(s))
    

In [12]:
# flatten the list
sentences = [y for x in sentences for y in x]

In [13]:
# remove punctuations, numbers and special characters
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")

# make alphabets lowercase
clean_sentences = [s.lower() for s in clean_sentences]

In [14]:
nltk.download('stopwords')# one time execution

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\shri1\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [15]:

from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [16]:
# function to remove stopwords
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new
    

  

In [17]:
# remove stopwords from the sentences
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

In [18]:
clean_sentences

['maria sharapova basically friends tennis players wta tour',
 'russian player problems openly speaking recent interview said really hide feelings much',
 'think everyone knows job',
 'courts court playing competitor want beat every single person whether locker room across net one strike conversation weather know next minutes go try win tennis match',
 'pretty competitive girl',
 'say hellos sending players flowers well',
 'uhm really friendly close many players',
 'lot friends away courts',
 'said really close lot players something strategic',
 'different men tour women tour',
 '',
 'think sport mean friends everyone categorized tennis player going get along tennis players',
 'think every person different interests',
 'friends completely different jobs interests met different parts life',
 'think everyone thinks tennis players greatest friends',
 'ultimately tennis small part',
 'many things interested',
 'basel switzerland ap roger federer advanced th swiss indoors final career beati

In [20]:
# Extract word vectors
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [21]:
sentence_vectors = []
for i in clean_sentences:
    if len(i) != 0:
        v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
    else:
         v = np.zeros((100,))
    sentence_vectors.append(v)
            
        


In [22]:
sentence_vectors

[array([ 5.14825583e-02,  1.10544682e-01,  6.94999397e-01,  1.89168096e-01,
        -9.58077684e-02,  3.20288986e-01,  2.70662010e-01,  5.42440832e-01,
        -3.05938005e-01, -1.56364068e-01,  3.70127618e-01,  8.09492469e-02,
         8.41393881e-03,  2.47571543e-01, -3.69342804e-01, -7.61044994e-02,
         8.08582604e-02,  2.30643645e-01, -2.70402402e-01,  5.13828397e-01,
        -6.12548441e-02,  3.87900352e-01,  1.03121363e-01,  7.72494674e-01,
         2.59960234e-01, -7.96069205e-02,  1.42143592e-01, -9.62644577e-01,
         7.54904330e-01,  6.03260659e-02, -4.58570123e-01,  2.36780301e-01,
         2.29152635e-01, -1.56453326e-01,  3.97632688e-01, -2.32720934e-02,
        -5.05520999e-01,  4.13252831e-01, -2.85759270e-01, -1.35231465e-01,
        -1.37098104e-01, -1.48972601e-01,  3.37537557e-01, -3.49540442e-01,
         1.53484434e-01, -2.33341649e-01, -1.98460802e-01, -1.27821520e-01,
         5.08063912e-01, -3.68636876e-01, -2.28472307e-01, -3.15306723e-01,
         1.3

In [23]:
len(sentence_vectors)

119

# The next step is to find similarities among the sentences. We will use cosine similarity to find similarity between a pair of sentences. Let's create an empty similarity matrix for this task and populate it with cosine similarities of the sentences.

In [24]:
# similarity matrix
sim_mat = np.zeros([len(sentences), len(sentences)])

In [25]:
from sklearn.metrics.pairwise import cosine_similarity

In [26]:
for i in range(len(sentences)):
    for j in range(len(sentences)):
         if i != j:
                sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]
                
    

In [27]:
sim_mat

array([[0.        , 0.64378333, 0.59156996, ..., 0.72776711, 0.58501214,
        0.68277591],
       [0.64378333, 0.        , 0.83267683, ..., 0.77598286, 0.79842889,
        0.8740449 ],
       [0.59156996, 0.83267683, 0.        , ..., 0.73326039, 0.75388014,
        0.81301224],
       ...,
       [0.72776711, 0.77598286, 0.73326039, ..., 0.        , 0.74423122,
        0.87150967],
       [0.58501214, 0.79842889, 0.75388014, ..., 0.74423122, 0.        ,
        0.78779912],
       [0.68277591, 0.8740449 , 0.81301224, ..., 0.87150967, 0.78779912,
        0.        ]])

In [28]:
import networkx as nx

nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

In [29]:
scores

{0: 0.008072651865276066,
 1: 0.008501993259365037,
 2: 0.0078119318398216,
 3: 0.009293791260564146,
 4: 0.007500319295916385,
 5: 0.008146814785247897,
 6: 0.008477413381565426,
 7: 0.008251000819455925,
 8: 0.008596957762357726,
 9: 0.008257144250233068,
 10: 0.0012695751770095795,
 11: 0.008860552409260931,
 12: 0.00808354331891815,
 13: 0.008156804650453691,
 14: 0.008443316877797879,
 15: 0.008556893043719335,
 16: 0.007812826653904838,
 17: 0.008071958049751223,
 18: 0.008406020961271342,
 19: 0.0088478922486596,
 20: 0.008860865186249187,
 21: 0.007421917083656914,
 22: 0.008223434004980818,
 23: 0.008991766451813816,
 24: 0.00846397039711992,
 25: 0.006701898152599973,
 26: 0.008232471647417278,
 27: 0.008913135600535109,
 28: 0.009061682997321345,
 29: 0.009093905696447463,
 30: 0.00924452161472398,
 31: 0.008994323963323616,
 32: 0.0072368691033659,
 33: 0.008709093081740912,
 34: 0.00891913055441074,
 35: 0.009097421351821915,
 36: 0.007715970774354713,
 37: 0.0088834520756

In [43]:
nx_graph[1]


AtlasView({0: {'weight': 0.6437833309173584}, 2: {'weight': 0.8326768279075623}, 3: {'weight': 0.8569855690002441}, 4: {'weight': 0.7176409363746643}, 5: {'weight': 0.8124058246612549}, 6: {'weight': 0.8420344591140747}, 7: {'weight': 0.8102104663848877}, 8: {'weight': 0.9047313928604126}, 9: {'weight': 0.7173464298248291}, 11: {'weight': 0.8244583010673523}, 12: {'weight': 0.8504345417022705}, 13: {'weight': 0.856580376625061}, 14: {'weight': 0.8253940939903259}, 15: {'weight': 0.7865604162216187}, 16: {'weight': 0.8493874073028564}, 17: {'weight': 0.5887117981910706}, 18: {'weight': 0.6343724727630615}, 19: {'weight': 0.70430588722229}, 20: {'weight': 0.761705756187439}, 21: {'weight': 0.5236850380897522}, 22: {'weight': 0.6297466158866882}, 23: {'weight': 0.7675076723098755}, 24: {'weight': 0.726353645324707}, 25: {'weight': 0.39574533700942993}, 26: {'weight': 0.6158713698387146}, 27: {'weight': 0.7406454086303711}, 28: {'weight': 0.7785936594009399}, 29: {'weight': 0.8151156902313

In [44]:
import matplotlib.pyplot as plt
plt.nx_graph

In [30]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

In [31]:
ranked_sentences

[(0.009293791260564146,
  "When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match."),
 (0.009260614592351134,
  'Major players feel that a big event in late November combined with one in January before the Australian Open will mean too much tennis and too little rest.'),
 (0.00924452161472398,
  'Speaking at the Swiss Indoors tournament where he will play in Sundays final against Romanian qualifier Marius Copil, the world number three said that given the impossibly short time frame to make a decision, he opted out of any commitment.'),
 (0.009213900380764764,
  '"I felt like the best weeks that I had to get to know players when I was playing were the Fed Cup weeks or the Olympic weeks, not necessarily during the tournaments.'),
 (0.0

In [32]:

# Specify number of sentences to form the summary
sn = 10

# Generate summary
for i in range(sn):
    print(ranked_sentences[i][1])
    


When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match.
Major players feel that a big event in late November combined with one in January before the Australian Open will mean too much tennis and too little rest.
Speaking at the Swiss Indoors tournament where he will play in Sundays final against Romanian qualifier Marius Copil, the world number three said that given the impossibly short time frame to make a decision, he opted out of any commitment.
"I felt like the best weeks that I had to get to know players when I was playing were the Fed Cup weeks or the Olympic weeks, not necessarily during the tournaments.
Currently in ninth place, Nishikori with a win could move to within 125 points of the cut for the eight-man event in London 